Load model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.utils import CustomObjectScope

path = f"/python_programming/phd/billboard_cascadenetwork/model/linknet"
#path = f"/python_programming/phd/billboard_cascadenetwork/model/unet"
#path = f"/python_programming/phd/billboard_cascadenetwork/model/adsegnet"

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

with CustomObjectScope({'iou': iou}):

    new_model = tf.keras.models.load_model(os.path.join(path, 'model_linknet.h5'))
    #new_model = tf.keras.models.load_model(os.path.join(path, 'billboard_unet_seg.hdf5'))
    #new_model = tf.keras.models.load_model(os.path.join(path, 'model_adsegnet.h5'))


video datasets:

1)'/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardonroad.mp4'

2) '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardtoday1.mp4'

3) '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardtoday2.mp4'

4) '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardtoday3.mp4'

save original video frames (128 x128)

In [ ]:
video_path = '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardonroad.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the frames per second (fps) and frame dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video Information: {width}x{height} pixels, {fps} fps, {total_frames} frames")

frame_imgs = []

frame_count = 1

while True:
    # Read a single frame
    ret, frame = cap.read()

    # Break the loop if no more frames are available
    if not ret:
      break

    # Preprocess the frame
    frame = cv2.resize(frame, (128, 128))  # Resize to match the input size of the model
    frame_img = frame

    frame_imgs.append(frame_img)

    # Print the result
    print(f'Frame {frame_count + 1}')

    '''
    plt.figure()
    plt.subplot(131)
    plt.imshow(frame_img)
    plt.axis('off')
    plt.title('Input image')
    '''

    cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/frames/billboardonroad/Frame_{frame_count}.jpg", frame_img)

    frame_count += 1

Video Information: 1070x720 pixels, 29 fps, 1573 frames
Frame 2
Frame 3
Frame 4
Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 16
Frame 17
Frame 18
Frame 19
Frame 20
Frame 21
Frame 22
Frame 23
Frame 24
Frame 25
Frame 26
Frame 27
Frame 28
Frame 29
Frame 30
Frame 31
Frame 32
Frame 33
Frame 34
Frame 35
Frame 36
Frame 37
Frame 38
Frame 39
Frame 40
Frame 41
Frame 42
Frame 43
Frame 44
Frame 45
Frame 46
Frame 47
Frame 48
Frame 49
Frame 50
Frame 51
Frame 52
Frame 53
Frame 54
Frame 55
Frame 56
Frame 57
Frame 58
Frame 59
Frame 60
Frame 61
Frame 62
Frame 63
Frame 64
Frame 65
Frame 66
Frame 67
Frame 68
Frame 69
Frame 70
Frame 71
Frame 72
Frame 73
Frame 74
Frame 75
Frame 76
Frame 77
Frame 78
Frame 79
Frame 80
Frame 81
Frame 82
Frame 83
Frame 84
Frame 85
Frame 86
Frame 87
Frame 88
Frame 89
Frame 90
Frame 91
Frame 92
Frame 93
Frame 94
Frame 95
Frame 96
Frame 97
Frame 98
Frame 99
Frame 100
Frame 101
Frame 102
Frame 103
Frame 104
Frame 105
Frame 106


Predict of video dataset

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


# Input video file path

video_path = '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardtoday1.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the frames per second (fps) and frame dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video Information: {width}x{height} pixels, {fps} fps, {total_frames} frames")

frame_imgs = []

frame_count = 1


def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

while True:
    # Read a single frame
    ret, frame = cap.read()

    # Break the loop if no more frames are available
    if not ret:
      break

    # Preprocess the frame
    frame = cv2.resize(frame, (128, 128))  # Resize to match the input size of the model
    frame_img = frame
    frame = frame / 255.0  # Normalize pixel values to be between 0 and 1

    # Expand dimensions to match the model's expected input shape
    frame = np.expand_dims(frame, axis=0)

    frame_imgs.append(frame_img)

    # Make predictions
    y_pred = new_model.predict(frame)
    y_predprob = np.squeeze(y_pred) * 255.0


    y_pred = y_pred[0] > 0.5
    y_pred = mask_parse(y_pred) * 255.0
    print('y_predprobshape:', y_predprob.shape)#(128, 128)
    print('y_pred shape:', y_pred.shape)#(128, 128, 3)


    print('prediction:', y_pred.shape)

    # Print the result
    print(f'Frame {frame_count + 1}')

    plt.figure()
    plt.subplot(131)
    plt.imshow(frame_img)
    plt.axis('off')
    plt.title('Input image')

    plt.subplot(132)
    cmap = plt.cm.jet
    plt.imshow(y_predprob, cmap = cmap)
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(y_pred)#, cmap="Greys_r")
    plt.axis('off')
    plt.title('Binary image')
    plt.show()

    #cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/unet/billboardonroad/frame_output_binary/Frame_{frame_count}.jpg", y_pred)
    #cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/linknet/billboardtoday1/frame_output_binary/Frame_{frame_count}.jpg", y_pred)
    cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/adsegnet/billboardtoday1/frame_output_binary/Frame_{frame_count}.jpg", y_pred)


    frame_count += 1

print('frame count:', len(frame_imgs))

Output hidden; open in https://colab.research.google.com to view.

draw bounding box

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


# Input video file path

video_path = '/python_programming/phd/billboard_cascadenetwork/datasets/video_dataset/Billboardtoday3.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the frames per second (fps) and frame dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video Information: {width}x{height} pixels, {fps} fps, {total_frames} frames")

frame_imgs = []

frame_count = 1


def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

while True:
    # Read a single frame
    ret, frame = cap.read()

    # Break the loop if no more frames are available
    if not ret:
      break

    # Preprocess the frame
    frame = cv2.resize(frame, (128, 128))  # Resize to match the input size of the model
    frame_img = frame
    frame = frame / 255.0  # Normalize pixel values to be between 0 and 1

    # Expand dimensions to match the model's expected input shape
    frame = np.expand_dims(frame, axis=0)

    frame_imgs.append(frame_img)

    # Make predictions

    # Make predictions
    y_pred = new_model.predict(frame)
    y_predprob = np.squeeze(y_pred) * 255.0

    y_pred = y_pred[0] > 0.5
    y_pred = mask_parse(y_pred) * 255.0
    print('y_predprobshape:', y_predprob.shape)#(128, 128)
    print('y_pred shape:', y_pred.shape)#(128, 128, 3)

    converted_image = y_pred.astype('uint8')
    cnts = cv2.cvtColor(converted_image, cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(cnts, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame_img, (x, y), (x + w, y + h), (0, 0, 255), 1)

    #cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/adsegnet/billboardonroad/bb_frame_output_binary/Frame_{frame_count}.jpg", frame_img)
    #cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/unet/billboardtoday3/bb_frame_output_binary/Frame_{frame_count}.jpg", frame_img)
    cv2.imwrite(f"/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/linknet/billboardtoday3/bb_frame_output_binary/Frame_{frame_count}.jpg", frame_img)


    # Print the result
    print(f'Frame {frame_count + 1}')

    plt.figure()
    plt.subplot(131)
    plt.imshow(frame_img)
    plt.axis('off')
    plt.title('Input image')

    plt.subplot(132)
    cmap = plt.cm.jet
    plt.imshow(y_predprob, cmap = cmap)
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(y_pred)#, cmap="Greys_r")
    plt.axis('off')
    plt.title('Binary image')
    plt.show()


    frame_count += 1

print('frame count:', len(frame_imgs))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def calculate_iou(mask1, mask2):
    """
    Calculate Intersection over Union (IoU) between two probability masks.
    """

    if mask1 is None or mask2 is None:
        raise ValueError("Invalid masks provided")

    # Compute intersection and union masks
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)

    # Calculate IoU
    intersection_area = np.sum(intersection)
    union_area = np.sum(union)

    if union_area == 0:
        iou = 0  # No overlapping pixels, IoU is 0
    else:
        iou = intersection_area / union_area

    return iou

def calculate_centroid(mask):
    """
    Calculate the centroid of a probability mask.
    """
    # Calculate the moments of the mask
    moments = cv2.moments(mask)

    # Calculate centroid coordinates
    if moments["m00"] != 0:
        centroid_x = int(moments["m10"] / moments["m00"])
        centroid_y = int(moments["m01"] / moments["m00"])
    else:
        centroid_x = -1
        centroid_y = -1

    return (centroid_x, centroid_y)

def calculate_area_ratio(mask, frame_shape):
    """
    Calculate the ratio of the area of the probability mask to the total area of the frame.
    """
    # Calculate area of the mask
    mask_area = np.sum(mask)

    # Calculate total area of the frame
    frame_area = frame_shape[0] * frame_shape[1] #128 X 128

    # Calculate area ratio
    area_ratio = mask_area / frame_area
    return area_ratio

# Example usage:
# Load probability masks for consecutive frames
mask_frame1 = cv2.imread("/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/fusionnet_v1/billboardonroad/frame_output_binary/Frame_1.jpg", cv2.IMREAD_GRAYSCALE) / 255.0  # Normalize to [0, 1]
mask_frame2 = cv2.imread("/python_programming/phd/billboard_cascadenetwork/test_results/video_frame/fusionnet_v1/billboardonroad/frame_output_binary/Frame_2.jpg", cv2.IMREAD_GRAYSCALE) / 255.0  # Normalize to [0, 1]

# Calculate IoU
iou = calculate_iou(mask_frame1, mask_frame2)
print("Intersection over Union (IoU):", iou)

# Calculate centroid for each mask
centroid_frame1 = calculate_centroid(mask_frame1)
centroid_frame2 = calculate_centroid(mask_frame2)
print("Centroid Frame 1:", centroid_frame1)
print("Centroid Frame 2:", centroid_frame2)

# Calculate area ratio for each mask
frame_shape = mask_frame1.shape  # Assuming both frames have the same shape
area_ratio_frame1 = calculate_area_ratio(mask_frame1, frame_shape)
area_ratio_frame2 = calculate_area_ratio(mask_frame2, frame_shape)
print("Area Ratio Frame 1:", area_ratio_frame1)
print("Area Ratio Frame 2:", area_ratio_frame2)


Intersection over Union (IoU): 0.9879317519766958
Centroid Frame 1: (90, 47)
Centroid Frame 2: (90, 47)
Area Ratio Frame 1: 0.13100897097120098
Area Ratio Frame 2: 0.13112984451593138
